In [18]:
import os
import time
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException


In [27]:

ean_list= ['8436028922536', '8436551456690', '3397701245487', '8435178361905']

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'


In [28]:
def bullet_points(all_bullets): 
    
    bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan') 
    
    if len(all_bullets.split("\n")) == 8: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8= [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 7: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7= [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 6: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6 = [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 5: 
        bullet1, bullet2, bullet3, bullet4, bullet5 = [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 4: 
        bullet1, bullet2, bullet3, bullet4 = [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 3: 
        bullet1, bullet2, bullet3, = [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 2: 
        bullet1, bullet2 = [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 1: 
        bullet1 = [str(e) for e in all_bullets.split("\n")]
    
    return bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8  


def asignar_variables_amazon(detalles_tecnicos, detalles_usuarios_amazon):
    title = price = bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 = main_image = marca = numero_de_modelo = valoracion = clasificacion = producto_desde = float('nan')
        
    for detalle_tecnico in detalles_tecnicos:
        if detalle_tecnico[0] == 'Marca':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Número de modelo':
            numero_de_modelo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Valoración media de los clientes':
            valoracion = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Clasificación en los más vendidos de Amazon':
            clasificacion = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Producto en Amazon.es desde':
            producto_desde = detalle_tecnico[1]
    
    for detalle_usuarios_amazon in detalles_usuarios_amazon:
        
        if detalle_usuarios_amazon[0] == 'Marca':
            marca = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Número de modelo':
            numero_de_modelo = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Valoración media de los clientes':
            valoracion = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Clasificación en los más vendidos de Amazon':
            clasificacion = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Producto en Amazon.es desde':
            producto_desde = detalle_usuarios_amazon[1]
    
    return marca, numero_de_modelo, valoracion, clasificacion, producto_desde


def initialize_amazon_info_df():
    
    amazon_info = pd.DataFrame(columns = ['item_name.value', 
                                                          'list_price.value_with_tax' , 'Bullet_point.value',
                                                          'Bullet_point#2.value','Bullet_point#3.value',
                                                          'Bullet_point#4.value', 'Bullet_point#5.value',
                                                          'Bullet_point#6.value','Bullet_point#7.value',
                                                          'main_image','Marca','Número de modelo',
                                                          'Valoración media de los clientes', 
                                                          'Clasificación en los más vendidos de Amazon',
                                                          'Producto en Amazon.es desde'
                                                         ])
    return amazon_info



In [31]:

def amazon(ean_list):
    
    amazon_info = initialize_amazon_info_df()
            
    browser = webdriver.Firefox()
    browser.get('https://www.amazon.es')
    
    time.sleep(3)
    
    for ean in ean_list:
        
        title = price = bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 = main_image = marca = numero_de_modelo = valoracion = clasificacion = producto_desde = float('nan')

        
        search = browser.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
        search.clear()
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)
        
        
        try:
    
            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//div[@class='a-section aok-relative s-image-fixed-height'])")
            action.move_to_element((click_image)).click().perform()
        
        except NoSuchElementException:
            
            print(f"Item {ean} not found")
        
        
        try: 
            
            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//div[@class='a-section aok-relative s-image-square-aspect'])")
            action.move_to_element((click_image)).click().perform()
            
         
        except NoSuchElementException:
            
            print(f"Item {ean} not found")
        
        
        try: 
            
            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//div[@class='a-section aok-relative s-image-tall-aspect'])")
            action.move_to_element((click_image)).click().perform()

        except NoSuchElementException:
            
            print(f"Item {ean} not found")
        
        
        time.sleep(3)
        
        
        try: 
            
            title = browser.find_elements_by_xpath('//h1[@id="title"]')[0].text
        
        except NoSuchElementException:
            
            print(f"title {ean} not found")
            
        except IndexError:
            
            print(f"title {ean} not found")
            
            
        try: 
            
            price = browser.find_elements_by_xpath('//span[@id="priceblock_ourprice"]')[0].text
            
        except NoSuchElementException:
            
            price = browser.find_elements_by_xpath('//span[@id="priceblock_dealprice"]')[0].text
            
        except NoSuchElementException:
            
            price = browser.find_elements_by_xpath('//span[@id="priceblock_saleprice"]')[0].text
            
        except NoSuchElementException:
            
            print(f"price {ean} not found")
        
        except IndexError:
            
            print(f"price {ean} not found")
            
            
        
        try:
            
            all_bullets = browser.find_elements_by_xpath('//div[@id="feature-bullets"]')[0].text
            bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8 = bullet_points(all_bullets)
        
        except NoSuchElementException:
            
            print(f"bullets {ean} not found")
        
        except IndexError:
            
            print(f"bullets {ean} not found")
            
        
        try: 
            
            main_image = browser.find_element_by_xpath('//img[@id="landingImage"]').get_attribute('src')
    
        except NoSuchElementException:
            
            print(f"main image {ean} not found")
        
        except IndexError:
            
            print(f"main image {ean} not found")
            
            
        try:
            
            url = str(browser.current_url)
            time.sleep(3)
            lista = pd.read_html(url, match='.+', flavor = None, header = None, index_col =None, skiprows=None, attrs=None, parse_dates=False, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True, displayed_only=True)
            time.sleep(3)

            detalles_tecnicos = lista[1].values.tolist()   
            detalles_usuarios_amazon = lista[2].values.tolist()  

            marca, numero_de_modelo, valoracion, clasificacion, producto_desde = asignar_variables_amazon(detalles_tecnicos, detalles_usuarios_amazon)

                    
            
        except urllib.error.HTTPError:
            print(f"detalles tecnicos {ean} not found")
            
        
        except NoSuchElementException:
            
            print(f"detalles tecnicos {ean} not found")
        
        except IndexError:
            
            print(f"detalles tecnicos {ean} not found")
                
        amazon_info.loc[ean] = [title, price, bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, main_image, marca, numero_de_modelo, valoracion, clasificacion, producto_desde]
        
        
    browser.close()

    return amazon_info


In [32]:
df= amazon(ean_list)

Item 8436028922536 not found
Item 8436028922536 not found
Item 8436028922536 not found
title 8436028922536 not found
price 8436028922536 not found
bullets 8436028922536 not found
main image 8436028922536 not found
detalles tecnicos 8436028922536 not found
Item 8436551456690 not found
Item 8436551456690 not found
Item 3397701245487 not found
Item 3397701245487 not found
Item 3397701245487 not found
title 3397701245487 not found
price 3397701245487 not found
bullets 3397701245487 not found
main image 3397701245487 not found
detalles tecnicos 3397701245487 not found
Item 8435178361905 not found
Item 8435178361905 not found
price 8435178361905 not found


In [33]:
df

,item_name.value,list_price.value_with_tax,Bullet_point.value,Bullet_point#2.value,Bullet_point#3.value,Bullet_point#4.value,Bullet_point#5.value,Bullet_point#6.value,Bullet_point#7.value,main_image,Marca,Número de modelo,Valoración media de los clientes,Clasificación en los más vendidos de Amazon,Producto en Amazon.es desde
8436028922536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8436551456690,"BLUALTA TV 32"" LED BLF32HD","114,46 €","Televisión LED de 32"" con TDT HD T2 integrada",Resolución HD READY 1366 x 768,"CONEXIÓN: AV, IN, USB: 2, HDMI: 2, COAXIAL, VG...","Incluye: TV, Mando a distancia, Manual de usua...",Brillo: 1000:1 Contraste: 200cd/m2 Salida audi...,› Ver más detalles,NaN,https://images-na.ssl-images-amazon.com/images...,Blualta,NaN,Sea la primera persona en escribir una opinión,nº23.449 en Electrónica (Ver el Top 100 en Ele...,5 de agosto de 2019
3397701245487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8435178361905,marckonfort Colchón Viscoelástico Ergo-Therapy...,NaN,Tapa superior: Realizada con tejido Strech de ...,Tapa inferior: Tejido 3D para facilitar la bue...,Plata-banda: Tejido 3D altamente transpirable ...,Núcleo: Bloque de espuma de 14 cm de alta dens...,Altura total de 18 cm. Ortopédico y sin muelles,› Ver más detalles,NaN,https://images-na.ssl-images-amazon.com/images...,marckonfort,NaN,"4,3 de 5 estrellas1.699 valoraciones de clientes",nº58.778 en Hogar y cocina (Ver el Top 100 en ...,3 de marzo de 2015


In [ ]:
Hace el try except por cada elemento, para el tema de la imagen buscar por titulo, siempre centrarse en la clase, es lo que no cambian. 